<a href="https://colab.research.google.com/github/thiagofdso/gemini-assistente-documento/blob/main/%5BImers%C3%A3o_IA_2%C2%AA_edi%C3%A7%C3%A3o%5D_C%C3%B3digo_Projeto_Assistente_Documento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.2/142.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 9.4 MB/s eta 0:00:00


In [18]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

# Configurar para exibir o conteúdo completo das colunas
pd.set_option('display.max_colwidth', None)  # Para pandas versões < 1.0.0, use -1 em vez de None

GOOGLE_API_KEY=""
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [12]:
# Caminho para o seu arquivo txt
caminho_do_arquivo = "Portaria_SGD_ME_5651_28_06_2022.txt"
# Referencia
# https://www.gov.br/governodigital/pt-br/contratacoes-de-tic/diretrizes-para-contratacao-de-servicos-de-desenvolvimento-e-manutencao-de-sistemas

# Inicializar uma lista vazia para armazenar os registros
registros = []

# Abrir o arquivo para leitura
with open(caminho_do_arquivo, 'r') as arquivo:
    # Ler cada linha do arquivo
    for linha in arquivo:
        # Remover espaços em branco e quebras de linha no início e no fim da linha
        linha_limpa = linha.strip()
        # Adicionar a linha limpa à lista de registros
        registros.append([linha_limpa])  # Cada linha é um registro em uma lista

# Criar um DataFrame a partir da lista de registros
# Cada registro da lista se torna uma linha no DataFrame, e há uma única coluna
df = pd.DataFrame(registros, columns=['Conteudo'])

# Adicionar uma nova coluna chamada 'Titulo' com o caminho_do_arquivo como valor
df['Titulo'] = caminho_do_arquivo

# Exibir as primeiras linhas do DataFrame para verificação
print(df.head())

                                            Conteudo  \
0  Boas práticas, vedações e orientações para con...   
1  As contratações de soluções de software e de s...   
2          As orientações estão divididas em partes:   
3  A. Contratações de serviços de desenvolvimento...   
4  B. Contratação de serviço de desenvolvimento e...   

                                Titulo  
0  Portaria_SGD_ME_5651_28_06_2022.txt  
1  Portaria_SGD_ME_5651_28_06_2022.txt  
2  Portaria_SGD_ME_5651_28_06_2022.txt  
3  Portaria_SGD_ME_5651_28_06_2022.txt  
4  Portaria_SGD_ME_5651_28_06_2022.txt  


In [7]:
model = "models/embedding-001"

In [9]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [13]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df.head()

,Conteudo,Titulo,Embeddings
0,"Boas práticas, vedações e orientações para con...",Portaria_SGD_ME_5651_28_06_2022.txt,"[0.015973205, -0.04109128, -0.033041187, 0.026..."
1,As contratações de soluções de software e de s...,Portaria_SGD_ME_5651_28_06_2022.txt,"[0.0136532625, -0.047040254, -0.04406508, 0.02..."
2,As orientações estão divididas em partes:,Portaria_SGD_ME_5651_28_06_2022.txt,"[0.0030821227, -0.025802929, -0.04306511, 0.00..."
3,A. Contratações de serviços de desenvolvimento...,Portaria_SGD_ME_5651_28_06_2022.txt,"[0.036816586, -0.019493774, -0.068442814, 0.03..."
4,B. Contratação de serviço de desenvolvimento e...,Portaria_SGD_ME_5651_28_06_2022.txt,"[0.0407735, -0.024845522, -0.07316768, 0.02592..."


In [27]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  #indice = np.argmax(produtos_escalares)
  # Usar np.argsort para obter os índices que ordenariam o array de produtos escalares em ordem ascendente
  indices_ordenados = np.argsort(produtos_escalares)

  # Para obter os índices em ordem descendente (do maior para o menor valor)
  top_20_indices_desc = indices_ordenados[-20:][::-1]

  return df.iloc[top_20_indices_desc]["Conteudo"]

In [30]:
consulta = "O que é recomendado para metrificar o esforço de desenvolvimento?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

65    25. Recomenda-se que, para remuneração das atividades do catálogo de serviços de portais web relativas à implantação, configuração e sustentação de portais web com Sistemas de Gerenciamento de Conteúdo, os órgãos desenvolvam uma métrica objetiva para mensuração dos resultados que utilizem em seu cálculo o fator esforço, desde que esteja sempre atrelado diretamente à entrega dos produtos ou resultados presentes no Catálogo, ou que realize um pagamento fixo mensal, com aplicação dos controles adequados, baseado em dados históricos, na experiência e na maturidade do órgão na execução dos serviços de desenvolvimento e sustentação de portais web com Sistemas de Gerenciamento de Conteúdo presentes no Catálogo.
46                                                                                                                                                                                                                                                                                      

In [25]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [32]:
prompt = f"Resuma o texto, sem adicionar informações que não façam parte do texto, considerando a pergunta {consulta}. Texto:  {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Para metrificar o esforço de desenvolvimento, recomenda-se:

* Utilizar uma métrica objetiva que inclua o fator esforço no cálculo, atrelada à entrega de produtos ou resultados.
* Incluir todas as atividades do ciclo de vida de desenvolvimento de software na métrica de pagamento.
* Adotar iterações curtas e entregas frequentes para diminuir riscos e melhorar o acompanhamento contratual.
* Capacitar servidores responsáveis pela auditoria e aferição das métricas de software.
* Especializar o Processo de Software considerando a complexidade das soluções.
* Avaliar a adoção do Roteiro de Métricas de Software do SISP e do Guia de Contagem de Pontos de Função do SISP.
* Elaborar um roteiro de contagem próprio para pontos não cobertos pelo Manual de Práticas de Contagem do IFPUG.
* Adotar critérios de teste e qualidade para soluções desenvolvidas.
